In [1]:
import torch
import torch.nn.functional as F
import llama_cpp
from tqdm import tqdm 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pickle
from synth import CrossEntropyDifferential

In [2]:
# instantiating phi to generate suggestions
master = llama_cpp.Llama(
      model_path="models/Phi-3.5-mini-instruct.Q8_0.gguf",
      n_gpu_layers=-1, 
      verbose=False,
      logits_all=False,
)

In [3]:
# gpt2 to evaluate cross-entropy and train
device = torch.device("cuda")
model_name = "gpt2"
slave = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)
tokenizer.pad_token = tokenizer.eos_token
slave.config.pad_token_id = slave.config.eos_token_id
cross_entropy_differential = CrossEntropyDifferential(slave, tokenizer, device)

In [40]:
def generate_Y_slave(
        model, 
        tokenizer, 
        seed="\n",
        how_many=1,
        max_new_tokens=16,
        temperature=1.2
    ):
    input = tokenizer(seed, return_tensors='pt', padding=True).to(device)
    output = model.generate(
            **input,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            num_return_sequences=how_many,
            do_sample=True, # uncomment to enable fancy sampling of the output distribution
            no_repeat_ngram_size=2,
            pad_token_id=model.config.eos_token_id
        )
    return tokenizer.batch_decode(output, skip_special_tokens=True)

def generate_Y_master(
        model, 
        seed="\n",
        max_new_tokens=16,
        temperature=1.2
    ):
    return model(
                seed,
                max_tokens = max_new_tokens,
                temperature=temperature
            )["choices"][0]["text"]

def generate_X(Y, max_tokens=30):
    S = "Write a TL;DR: of the following text in exactly 5 words. No more, no less. Separate words with spaces. End with a . after writing it."
    prompt = f"""<|system|>{S}<|end>
    <|user|>{Y}<|end|>
    <|assistant|>"""
    return master(prompt, temperature=4, stop=["."], max_tokens=max_tokens)["choices"][0]["text"]

def ced(Y,X):
    return cross_entropy_differential(X, Y, diff=True)

In [21]:
for i in range(5):
    Y = generate_Y_slave(slave, tokenizer, max_new_tokens=130, temperature=1.2, how_many=1)
    for y in Y: 
        print(f"\n\nNew phrase ------------------\n {y}")



New phrase ------------------
 
: The United States is sending some $80 billion per year to Iraq. As long as there are sanctions, what can we do with these money when the regime's economy continues unabated and the economy is at the lowest point?

(2)(A) This bill contains the following provisions:
​(1) Nothing written nor approved by the President or any other elected or appointed director of the United Nations, or other such unelected person in the executive branch of government acting for the purposes specified by these two provisions shall be adopted by Congress so soon as is practicable that it is likely that future appropriations are not being appropriated prior to the date


New phrase ------------------
 
Bathrooms open on the first Wednesday on weekdays 8-9 (1pm). At other bars one of every two Saturdays or Sundays the weekend and during the day there will be a "special lunch break."

There is always a line for one hour straight between the following two days. However, durin

In [8]:
for i in range(20):
    Y = generate_Y_master(master, max_new_tokens=80, temperature=1.2)
    print(f"\n\nNew phrase ------------------\n {Y}")



New phrase ------------------
 
A:

For every word in the file (each line is considered a "word") we read the first letter and put it into a map of the letter-to-occurrence count.
Map<Character, Integer> frequencyMap = new HashMap<Character, Integer>();
for (String word : lines) {
   for (char c : word.toCharArray())


New phrase ------------------
 
\item \textbf{Ratio}

Ratio refers to the relationship between two quantities, showing how many times one quantity is contained within the other. For example, if we have 10 apples and 5 oranges, the ratio of apples to oranges would be 2:1. 

\begin{itemize}
    \item The concept of


New phrase ------------------
 
  **Question 1:** What was the relationship between John, Mary, and Sarah based on the text provided? 



  **Answer 1:** The text does not provide specific information regarding the relationship between John, Mary, and Sarah. 



 **Question 2:** Did John and Mary share a common interest in gardening? 



New phrase ---------

### Notes

Given the type of spontaneous generation of GPT2 and Phi, I'd go with GPT2. Phi generates Q: A: type of spontaneous output or even code. Probably this reflects the kind of data is has been trained on, so I'll just go with GPT2. 

We'll use GPT2 to generate random Ys, then we'll Phi to generate Xs. Then we will use this dataset to train a new version of GPT2. 

In [23]:
Y = generate_Y_slave(slave, tokenizer, max_new_tokens=240, temperature=1.2)
X = generate_X(Y[0])

In [43]:
# print(f"Y: {Y[0]}\n")
# print(f"X: {X}")
best = ("",0)
for _ in tqdm(range(1000)):
    X = generate_X(Y[0], max_tokens=16)
    rank = ced(Y[0], X)
    if rank < best[1]:
        best = (X, rank)
print(f"Best: {best}")

100%|██████████| 1000/1000 [03:31<00:00,  4.73it/s]

Best: (' Custom cards $25 Amazon RCT 3 years, RCRL, deck', tensor(-0.1230, device='cuda:3'))


In [47]:
# print(f"Y: {Y[0]}\n")
# print(f"X: {X}")
best = ("",0)
for _ in tqdm(range(100)):
    X = generate_X(Y[0], max_tokens=16)
    rank = ced(Y[0], X)
    if rank < best[1]:
        best = (X, rank)
print(f"Best: {best}")

100%|██████████| 100/100 [00:21<00:00,  4.66it/s]

Best: (' Buy custom cards online for $25, missing Amazon deck info', tensor(-0.0992, device='cuda:3'))


: 

In [ ]:
n_gen = 200
gamma = -0.75
new_suggestions = []

barra = tqdm(total=n_gen)

while len(new_suggestions) < n_gen:
    Y = generate_Y_slave(slave, tokenizer, max_new_tokens=240, temperature=1.2)
    X = generate_X(Y)

In [ ]:


n_gen = 5000
gamma = -0.4
new_suggestions = []

barra = tqdm(total=n_gen)

while len(new_suggestions) < n_gen:    
    suggestion = generate_suggestion()
    rank = rank_suggestion(suggestion)
    if rank < gamma:
        barra.update(1)
        new_suggestions.append({f"{Y}::TLDR::{suggestion}":rank.item()})

barra.close()
phi.close()

with open('pizza_synthdataset.pkl', 'wb') as f:
    pickle.dump(new_suggestions, f)
